In [3]:

start_chunk=[0, 4, 9, 12, 18, 23, 26, 30, 35, 40, 44, 49, 54, 58, 63, 67, 71, 77, 81, 86, 91, 94, 98, 103, 106, 109, 113, 117, 122, 128]
end_chunk=[2, 8, 11, 17, 22, 25, 27, 33, 39, 43, 46, 52, 57, 61, 66, 70, 74, 80, 84, 88, 93, 97, 100, 105, 107, 111, 116, 120, 127, 130]
start_end = sorted(start_chunk+end_chunk)

max_len=134
i=0


In [4]:
print (start_end)

[0, 2, 4, 8, 9, 11, 12, 17, 18, 22, 23, 25, 26, 27, 30, 33, 35, 39, 40, 43, 44, 46, 49, 52, 54, 57, 58, 61, 63, 66, 67, 70, 71, 74, 77, 80, 81, 84, 86, 88, 91, 93, 94, 97, 98, 100, 103, 105, 106, 107, 109, 111, 113, 116, 117, 120, 122, 127, 128, 130]


In [33]:
start_end = [1,3,5,7,9,12]
max_len = 13
len_mapping=[]
if start_end[0]==0:
    len_mapping.append(start_end[1]-start_end[0])
if start_end[0]!=0:
    len_mapping =len_mapping +[1]*(start_end[0])
    len_mapping.append (start_end[1]-start_end[0])
i=1
while True:

    if i%2 ==0:
        len_mapping.append(start_end[i+1]-start_end[i])
        
    if i%2 !=0:
        if start_end[i+1]-start_end[i] ==1:
            i=i+1
            continue

        if start_end[i+1]-start_end[i] >1:
            len_mapping =len_mapping +[1]*(start_end[i+1]-start_end[i])
    i=i+1
    print (i)
    if i ==len (start_end) -1:
        break
if start_end[i]==max_len:
    None
if start_end [i]<max_len:
    len_mapping =len_mapping +[1]*(max_len-start_end[i])

print (len_mapping)

2
3
4
5
[1, 2, 1, 1, 2, 1, 1, 3, 1]


In [44]:
def id_len_mapping(start_chunk,end_chunk,max_pos):
    start_end =sorted(start_chunk+end_chunk)
    print (start_end)
    len_mapping=[]
    if start_end[0]==0:
        len_mapping.append(start_end[1]-start_end[0])
    if start_end[0]!=0:
        len_mapping =len_mapping +[1]*(start_end[0])
        len_mapping.append (start_end[1]-start_end[0])
    i=1
    while True:

        if i%2 ==0:
            len_mapping.append(start_end[i+1]-start_end[i])

        if i%2 !=0:
            if start_end[i+1]-start_end[i] ==1:
                i=i+1
                continue

            if start_end[i+1]-start_end[i] >1:
                len_mapping =len_mapping +[1]*(start_end[i+1]-start_end[i])
        i=i+1
        print (i)
        if i ==len (start_end) -1:
            break
    if start_end[i]==max_pos:
        None
    if start_end [i]<max_pos:
        len_mapping =len_mapping +[1]*(max_pos-start_end[i])
    return len_mapping

In [45]:
len_mapping = id_len_mapping([0,4],[3,9],10)

[0, 3, 4, 9]
3


In [46]:
print (len_mapping)

[3, 5, 1]


In [52]:
import en_core_web_md
from process import parse_sentence
from tqdm import tqdm
from transformers import AutoTokenizer, BertModel, GPT2Model
from mapper import Map, deduplication
nlp = en_core_web_md.load()
import torch
import numpy as np
from utils import compress_attention, create_mapping, BFS, build_graph, is_word
from multiprocessing import Pool
import spacy
import en_core_web_md
import torch
from transformers import AutoTokenizer, BertModel, GPT2Model
from constant import invalid_relations_set
from utils import compress_attention,build_graph
language_model = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(language_model)
encoder = BertModel.from_pretrained(language_model)
encoder.eval()
use_cuda =False
threshold = 0.01
# def process_matrix(attentions, layer_idx = -1, head_num = 0, avg_head=False, trim=True, use_cuda=True):
#     if avg_head:
#         if use_cuda:
#             attn =  torch.mean(attentions[0][layer_idx], 0).cpu()
#         else:
#             attn = torch.mean(attentions[0][layer_idx], 0)
#         attention_matrix = attn.detach().numpy()
#     else:
#         attn = attentions[0][layer_idx][head_num]
#         if use_cuda:
#             attn = attn.cpu()
#         attention_matrix = attn.detach().numpy()

#     attention_matrix = attention_matrix[1:-1, 1:-1]

#     return attention_matrix
with open ('examples/patent_example.txt','r') as f:
    num_file = sum([1 for i in open('examples/patent_example.txt', "r")])
    #tqdm summmarization
    for idx, line in enumerate(tqdm (f,total=num_file)):
        sentence  = line.strip()

100%|██████████| 1/1 [00:00<00:00, 2135.59it/s]


In [53]:
print (sentence)

The invention relates to a roadable aircraft vehicle and related systems. An example roadable aircraft vehicle includes a vehicle drive system including an engine and gearbox selectively engageable with an automotive driveline  and at least one propeller , a user interface including a display  for controlling the drive system  in an automotive mode including a steering wheel and in a flight mode including a control stick , a control system for switching between the flight mode and the automotive mode, and a system  for locking the propeller during the automotive mode. The invention also relates to aircraft systems and elements such as an airfoil  having a nominal profile, a folding wing , and an occupant crash protection system for an aircraft .


In [54]:
import re
new_s = re.sub(' +', ' ', sentence)

In [55]:
print (new_s)

The invention relates to a roadable aircraft vehicle and related systems. An example roadable aircraft vehicle includes a vehicle drive system including an engine and gearbox selectively engageable with an automotive driveline and at least one propeller , a user interface including a display for controlling the drive system in an automotive mode including a steering wheel and in a flight mode including a control stick , a control system for switching between the flight mode and the automotive mode, and a system for locking the propeller during the automotive mode. The invention also relates to aircraft systems and elements such as an airfoil having a nominal profile, a folding wing , and an occupant crash protection system for an aircraft .
